## Build Model

### Using the python API.

Here we are going to build a model using the HydroMT-FIAT python API.

In [ ]:
# Imports
import tempfile
from pathlib import Path

from hydromt._utils import log

from hydromt_fiat import FIATModel
from hydromt_fiat.data import fetch_data

Lets make sure that there is a directory to build to.

Also ensure we have some data to build with, hydromt_fiat comes with a small to function to fetch a small dataset to build with.

In [ ]:
# Use the built-in tempfile package
temp_dir = tempfile.TemporaryDirectory(delete=False)
p = Path(temp_dir.name)

## Get the build data
global_dir = fetch_data("global-data")
local_dir = fetch_data("test-build-data")

Now lets setup the Model.

In [ ]:
# Create a model in write mode.
log._setuplog()
model = FIATModel(
    root=p,
    mode="w+",
    data_libs=[
        Path(global_dir, "data_catalog.yml"),
        Path(local_dir, "data_catalog.yml"),
    ],
)

Next step is to update the configurations and add a region for the model to know to collect data in.

Luckily a region vector file is incorporated in the small build data directory.

In [ ]:
# Add some of the configurations
model.setup_config(**{"model.type": "geom", "output.path": "output"})

# Add a region to build with
model.setup_region(Path(local_dir, "region.geojson"))

Lets verify the presence of our region, which is crucial for building our hazard and exposure data.

In [ ]:
model.region

All seems in order, therefore we can move on to setting up the FIAT model data.

Let's start with the vulnerability, as this data is crucial for the exposure data.

In [ ]:
# Call the setup method of the component
model.vulnerability.setup(
    vulnerability_fname="jrc_curves",
    vulnerability_linking_fname="jrc_curves_link",
    unit="m",
    continent="europe",  # A keyword argument to select data from the
    # vulnerability_fname dataset
)

This should result in damage curve data in our vulnerability component, lets see.

In [ ]:
model.vulnerability.data.curves

Simply done.

Let's continue with the hazard data.

Hazard data is easily setup from a raster-based datasource.

In [ ]:
# Call the setup method from the hazard component
model.hazard.setup(
    hazard_fnames="flood_event",
    hazard_type="water depth",
)

Let's take a look a the current state of the hazard data after settting it up 
with the setup method.

In [ ]:
model.hazard.data